# 군집별 특성 확인
군집별 user_spec, log_result의 feature에 대해 군집별 특성을 분석함
- user_spec 대푯값(min, max, std) / age, 입사년차, 월별만 해보기 —> 두드러지는 정보가 있는가
- log_result → 변형한 변수에 대한 통계값. 그 변수에 대한
- #,#,#(is_applied한 군집) / #, #(is_applied 안 한 군집) label간 데이터 분리
- kdeplot → 분포 해석
- 군집 해석 및 마켓팅 아이디어 생각필요..!

# 1. user_spec데이터 군집별 분석

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import gzip
from IPython.display import display
from tqdm import tqdm

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
# matplotlib.rcParams['axes.unicode_minus'] = False
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

In [2]:
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})

In [3]:
df = pd.read_csv('../processed/5_4_cluster_except.csv')

In [5]:
df0 = df[df['label'] == 0]
df1 = df[df['label'] == 1]
df2 = df[df['label'] == 2]
df3 = df[df['label'] == 3]
df4 = df[df['label'] == 4]

In [ ]:
df0.columns

In [21]:
df['income_type']

0        0 
1        0 
2        2 
3        2 
4        1 
         ..
107606   0 
107607   0 
107608   2 
107609   0 
107610   0 
Name: income_type, Length: 107611, dtype: float64

In [12]:
pd.options.display.float_format = '{:.0f}'.format

In [ ]:
df0.groupby('income_type')['desired_amount'].mean()

In [ ]:
df1.groupby('income_type')['desired_amount'].mean()

In [ ]:
df2.groupby('income_type')['desired_amount'].mean()

In [ ]:
df3.groupby('income_type')['desired_amount'].mean()

In [ ]:
df4.groupby('income_type')['desired_amount'].mean()

In [ ]:
earned_income = df[df['income_type'] == 5.0]
earned_income2 = 
private = df[df['income_type'] == 5.0]
private

In [ ]:
'income_type'

In [4]:
df.shape

(107611, 47)

In [ ]:
df.head()

In [6]:
df.groupby('label')['user_id'].count()

label
0     9029
1    54504
2     7762
3     8821
4    27495
Name: user_id, dtype: int64

In [6]:
df.shape

(107611, 47)

In [ ]:
df.groupby(['label', 'is_applied'])['user_id'].count()

In [11]:
df0 = df[df['label'] == 0]
df1 = df[df['label'] == 1]
df2 = df[df['label'] == 2]
df3 = df[df['label'] == 3]
df4 = df[df['label'] == 4]

In [ ]:
not_applied = pd.concat([df2, df3])
applied = pd.concat([df0, df1, df4])
display(not_applied.head())
display(applied.head())

In [ ]:
#not_applied, applied
fig, ax=plt.subplots(nrows=1, ncols=2, figsize=(20,5), constrained_layout=True)
sns.kdeplot(x = 'is_applied', data = not_applied, hue = 'label', shade = True, palette=['#BB3A32', '#8D6AB7'] , ax = ax[0])
sns.kdeplot(x = 'is_applied', data = applied, hue = 'label', shade = True, palette=['#E58537', '#4A76AF', '#5A9D40'], ax = ax[1])
#plt.show()
plt.savefig('../output/5_91_is_applied.png')

In [ ]:
pd.DataFrame(applied.groupby(['label', 'is_applied'])['user_id'].count())

In [ ]:
pd.DataFrame(not_applied.groupby(['label', 'is_applied'])['user_id'].count())

In [39]:
df.columns

Index(['purpose', 'credit_score', 'desired_amount',
       'purpose_by_income_type_SWITCHLOAN', 'insert_hour', 'existing_loan_amt',
       'existing_loan_amt_by_credit_score_cut_quantile_first',
       'existing_loan_cnt', 'income_type', 'desired_amount_by_income_type_std',
       'existing_loan_amt_by_purpose_max', 'insert_day', 'gender', 'age',
       'insert_month', 'yearly_income_by_houseown_type_mean',
       'purpose_by_income_type_BUYCAR', 'employment_type', 'is_applied',
       'CompleteIDCertification_first', 'GetCreditInfo_first',
       'UseDSRCalc_first', 'UseLoanManage_first', 'UsePrepayCalc_first',
       'is_applied_Y_first', 'CompleteIDCertification_last',
       'GetCreditInfo_last', 'UseDSRCalc_last', 'UseLoanManage_last',
       'UsePrepayCalc_last', 'is_applied_Y_last', 'timedelta_max',
       'timedelta_min', 'timedelta_mean', 'timedelta_std',
       'timedelta_quantile_first', 'timedelta_quantile_second',
       'timedelta_quantile_third', 'CompleteIDCertification

In [6]:
cluster_shap_features=['is_applied_Y_last', 'timedelta_max', 
                       'desired_amount_by_income_type_std', 
                       'purpose_by_income_type_BUYCAR', 'income_type', 
                       'purpose_by_income_type_SWITCHLOAN', 
                       'timedelta_min', 'existing_loan_amt', 
                       'CompleteIDCertification_count', 
                       'gender', 'existing_loan_cnt', 
                       'CompleteIDCertification_first', 
                       'GetCreditInfo_count', 
                       'CompleteIDCertification_last', 
                       'credit_score', 'timedelta_std', 
                       'GetCreditInfo_last', 
                       'UseLoanManage_count', 
                       'timedelta_quantile_third', 
                       'insert_hour', 'is_applied']

In [7]:
category = ['purpose', 'insert_day', 'gender', 'income_type',
            'insert_month', 'employment_type']

user_spec = ['credit_score', 'desired_amount',
       'purpose_by_income_type_SWITCHLOAN', 'existing_loan_amt',
       'existing_loan_amt_by_credit_score_cut_quantile_first',
       'existing_loan_cnt', 'desired_amount_by_income_type_std',
       'existing_loan_amt_by_purpose_max', 'insert_day', 'age',
       'yearly_income_by_houseown_type_mean',
       'purpose_by_income_type_BUYCAR']

log_result = ['UseDSRCalc_first', 'UseLoanManage_first', 'UsePrepayCalc_first',
       'is_applied_Y_first', 'CompleteIDCertification_last',
       'GetCreditInfo_last', 'UseDSRCalc_last', 'UseLoanManage_last',
       'UsePrepayCalc_last', 'is_applied_Y_last', 'timedelta_max',
       'timedelta_min', 'timedelta_mean', 'timedelta_std',
       'timedelta_quantile_first', 'timedelta_quantile_second',
       'timedelta_quantile_third', 'CompleteIDCertification_count',
       'GetCreditInfo_count', 'UseDSRCalc_count', 'UseLoanManage_count',
       'UsePrepayCalc_count']

## 1) category변수에 대한 plot

In [138]:
#category변수가 각각 어떤 요소로 이루어져있는지 알아내기
import joblib

with open('../final_cluster/ordinal_encoder.pickle', 'rb') as f:
    ordinal = joblib.load(f)

In [156]:
ordinal.categories_

[array(['EARNEDINCOME', 'EARNEDINCOME2', 'FREELANCER', 'OTHERINCOME',
        'PRACTITIONER', 'PRIVATEBUSINESS'], dtype=object),
 array(['contract_worker', 'daily_worker', 'etc', 'regular_worker'],
       dtype=object),
 array(['family', 'own', 'rent', 'spouse'], dtype=object),
 array(['BUSINESS', 'BUYCAR', 'BUYHOUSE', 'ETC', 'HOUSEDEPOSIT', 'INVEST',
        'LIVING', 'SWITCHLOAN'], dtype=object)]

### 1-1) income type

In [135]:
# income_type에 대한 각 군집의 막대그래프
from scipy.stats import mode
mode_list = (mode(df0['income_type']), mode(df1['income_type']), mode(df2['income_type']),mode(df3['income_type']), mode(df4['income_type']))
mode_list

(ModeResult(mode=array([1.]), count=array([5167])),
 ModeResult(mode=array([0.]), count=array([49574])),
 ModeResult(mode=array([0.]), count=array([7583])),
 ModeResult(mode=array([5.]), count=array([8606])),
 ModeResult(mode=array([0.]), count=array([22690])))

In [15]:
idf0 = df0.groupby('income_type')['user_id'].count().reset_index()
idf1 = df1.groupby('income_type')['user_id'].count().reset_index()
idf2 = df2.groupby('income_type')['user_id'].count().reset_index()
idf3 = df3.groupby('income_type')['user_id'].count().reset_index()
idf4 = df4.groupby('income_type')['user_id'].count().reset_index()

In [ ]:
fig, ax=plt.subplots(nrows=3, ncols=2, figsize=(30,30), constrained_layout=True)
sns.barplot(x = 'income_type', y = 'user_id', data = idf0, palette=['#4A76AF', '#E58537', '#5A9D40','#BB3A32', '#8D6AB7'] , ax = ax[0, 0])
sns.barplot(x = 'income_type', y = 'user_id', data = idf1, palette=['#4A76AF', '#E58537', '#5A9D40','#BB3A32', '#8D6AB7'] , ax = ax[0, 1])
sns.barplot(x = 'income_type', y = 'user_id', data = idf2, palette=['#4A76AF', '#E58537', '#5A9D40','#BB3A32', '#8D6AB7'] , ax = ax[1, 0])
sns.barplot(x = 'income_type', y = 'user_id', data = idf3, palette=['#4A76AF', '#E58537', '#5A9D40','#BB3A32', '#8D6AB7'] , ax = ax[1, 1])
sns.barplot(x = 'income_type', y = 'user_id', data = idf4, palette=['#4A76AF', '#E58537', '#5A9D40','#BB3A32', '#8D6AB7'] , ax = ax[2, 0])
plt.yscale('log')
plt.rc('font', size=10)
plt.rc('xtick', labelsize=5)  # x축 눈금 폰트 크기 
plt.rc('ytick', labelsize=5)  # y축 눈금 폰트 크기
plt.show()

대출자의 income type은 <br>
0번 군집의 경우 ####, ####<br>
1번 군집은 ####,<br>
2번 군집은 ####,<br>
3번 군집은 ####,<br>
4번 군집은 ####<br>
주로 ####인 사람들로 나타남. 그 중 눈에 띄는 군집은 3번 군집으로 거의 모든 사람이 ####에 속함.

### 1-2) purpose

In [11]:
pdf0 = df0.groupby('purpose')['user_id'].count().reset_index()
pdf1 = df1.groupby('purpose')['user_id'].count().reset_index()
pdf2 = df2.groupby('purpose')['user_id'].count().reset_index()
pdf3 = df3.groupby('purpose')['user_id'].count().reset_index()
pdf4 = df4.groupby('purpose')['user_id'].count().reset_index()

In [ ]:
pdf1

In [218]:
len(df[df['purpose'] == 1])

232

In [ ]:
fig, ax=plt.subplots(nrows=3, ncols=2, figsize=(30,30), constrained_layout=True)
sns.barplot(x = 'purpose', y = 'user_id', data = pdf0, palette='viridis' , ax = ax[0, 0])
sns.barplot(x = 'purpose', y = 'user_id', data = pdf1, palette='viridis' , ax = ax[0, 1])
sns.barplot(x = 'purpose', y = 'user_id', data = pdf2, palette='viridis' , ax = ax[1, 0])
sns.barplot(x = 'purpose', y = 'user_id', data = pdf3, palette='viridis' , ax = ax[1, 1])
sns.barplot(x = 'purpose', y = 'user_id', data = pdf4, palette='viridis' , ax = ax[2, 0])
#plt.yscale('log')
plt.rc('font', size=20)
plt.rc('xtick', labelsize=50)  # x축 눈금 폰트 크기 
plt.rc('ytick', labelsize=50)  # y축 눈금 폰트 크기
plt.show()

대출목적은 대체적으로 (#, ###), (#, ####)인 것으로 밝혀졌음. 그 다음 후순위로는 (#, ###), (#, ###)가 있음.

### 1-3) gender

In [ ]:
df.groupby('gender')['user_id'].count()

In [ ]:
gdf0 = df0.groupby('gender')['user_id'].count().reset_index()
gdf1 = df1.groupby('gender')['user_id'].count().reset_index()
gdf2 = df2.groupby('gender')['user_id'].count().reset_index()
gdf3 = df3.groupby('gender')['user_id'].count().reset_index()
gdf4 = df4.groupby('gender')['user_id'].count().reset_index()

display(gdf0)
display(gdf1)
display(gdf2)
display(gdf3)
display(gdf4)

In [ ]:
fig, ax=plt.subplots(nrows=3, ncols=2, figsize=(30,30), constrained_layout=True)
sns.barplot(x = 'gender', y = 'user_id', data = gdf0, palette='viridis' , ax = ax[0, 0])
sns.barplot(x = 'gender', y = 'user_id', data = gdf1, palette='viridis' , ax = ax[0, 1])
sns.barplot(x = 'gender', y = 'user_id', data = gdf2, palette='viridis' , ax = ax[1, 0])
sns.barplot(x = 'gender', y = 'user_id', data = gdf3, palette='viridis' , ax = ax[1, 1])
sns.barplot(x = 'gender', y = 'user_id', data = gdf4, palette='viridis' , ax = ax[2, 0])
#plt.yscale('log')
plt.rc('font', size=20)
plt.rc('xtick', labelsize=50)  # x축 눈금 폰트 크기 
plt.rc('ytick', labelsize=50)  # y축 눈금 폰트 크기
plt.show()

#####수가 있는 #번, #번, #번 군집에 대해서는 전체적인 경향성과 비슷하게 ##보다 ##이 많은 수로 나타남.<br>
<b>이와 달리, #번 군집의 경우에는 대부분이 ##으로(####/####) 특징적인 분포가 나타남.<br></b>
#번 군집의 경우 ####없지만 전체적인 경향성과 유사함

### 1-4) 나이대 박스플롯 그리기

In [ ]:
fig,ax = plt.subplots(1,2)
sns.violinplot(y='age',x='label',data=df.query('gender==0'), ax=ax[0])
sns.violinplot(y='age',x='label',data=df.query('gender==0'), ax=ax[1])

plt.suptitle('Age Boxplot')
ax[0].set_xticks([0, 1, 2, 3, 4])
ax[1].set_yticks([0, 1, 2, 3, 4])
plt.rc('axes', labelsize=20)   # x,y축 label 폰트 크기
plt.rc('xtick', labelsize=20)  # x축 눈금 폰트 크기 
plt.rc('ytick', labelsize=20)  # y축 눈금 폰트 크기
plt.show()

In [ ]:
a = sns.violinplot(data = df, x = 'label', 
               y='age', hue = 'gender', hue_order = [0, 1],
              split = True)
plt.savefig('../output/5_92_age_violin.png')

### user_spec에 대한 그 외의 변수의 "대푯값 뽑은 후 막대그래프로 표현

In [ ]:
# user_spec에 대한 대푯값 뽑기
pd.options.display.float_format = '{:.0f}'.format
grouped_user= df.groupby(['label'])[['desired_amount',
       'existing_loan_amt', 'desired_amount_by_income_type_std']]
grouped_user3 = grouped_user.agg(['mean', 'median']).reset_index()
grouped_user3

0번 군집의 희망대출금액은 약 ####만원, 기대출금액의 평균은 약 ####만원, <br>

1번 군집의 희망대출금액은 약 ####만원, 기대출금액의 평균은 약 ####만원, <br>

2번 군집의 희망대출금액은 약 ####만원, 기대출금액의 평균은 약 ####만원, <br>

3번 군집의 희망대출금액은 약 ####만원, 기대출금액의 평균은 약 ####만원, <br>

4번 군집의 희망대출금액은 약 ####만원, 기대출금액의 평균은 약 ####만원, <br>


In [ ]:
# user_spec에 대한 대푯값 뽑기
pd.options.display.float_format = '{:.0f}'.format
grouped_user= df.groupby(['label'])[['credit_score',
       'purpose_by_income_type_SWITCHLOAN',
       'existing_loan_amt_by_credit_score_cut_quantile_first',
       'existing_loan_cnt','existing_loan_amt_by_purpose_max', 
       'insert_day', 'age', 'yearly_income_by_houseown_type_mean',
       'purpose_by_income_type_BUYCAR']]
grouped_user3 = grouped_user.agg(['mean', 'median']).reset_index()
grouped_user3

In [ ]:
# MultiIndex 컬럼을 평탄화 하는 함수
def flat_cols(df):
    df.columns = ['/'.join(x) for x in df.columns.to_flat_index()]
    return df

# 컬럼별 다른 통계량 산출
grouped_user2 = df.groupby('label').agg({'desired_amount': ['mean', 'median'],
                       'existing_loan_amt': ['mean', 'median'],
                       'desired_amount_by_income_type_std': ['mean', 'median']}).pipe(flat_cols).reset_index()
grouped_user2

In [ ]:
# 어떻게 하면 plot을 구할 수 있지?
fig, ax=plt.subplots(nrows=3, ncols=2, figsize=(30,30), constrained_layout=True)
sns.barplot(x = 'label', y = 'desired_amount/mean', data = grouped_user2, palette=['#4A76AF', '#E58537', '#5A9D40','#BB3A32', '#8D6AB7'] , ax = ax[0, 0])
sns.barplot(x = 'label', y = 'desired_amount/median', data = grouped_user2, palette=['#4A76AF', '#E58537', '#5A9D40','#BB3A32', '#8D6AB7'] , ax = ax[0, 1])
sns.barplot(x = 'label', y = 'existing_loan_amt/mean', data = grouped_user2, palette=['#4A76AF', '#E58537', '#5A9D40','#BB3A32', '#8D6AB7'] , ax = ax[1, 0])
sns.barplot(x = 'label', y = 'existing_loan_amt/mean', data = grouped_user2, palette=['#4A76AF', '#E58537', '#5A9D40','#BB3A32', '#8D6AB7'] , ax = ax[1, 1])
sns.barplot(x = 'label', y = 'desired_amount_by_income_type_std/mean', data = grouped_user2, palette=['#4A76AF', '#E58537', '#5A9D40','#BB3A32', '#8D6AB7'] , ax = ax[2, 0])
sns.barplot(x = 'label', y = 'desired_amount_by_income_type_std/median', data = grouped_user2, palette=['#4A76AF', '#E58537', '#5A9D40','#BB3A32', '#8D6AB7'] , ax = ax[2, 1])
plt.yscale('log')
plt.rc('font', size=20)
plt.rc('axes', labelsize=20)   # x,y축 label 폰트 크기
plt.show()

In [ ]:
# purpose_by_income_type_SWITCHLOAN
#not_applied, applied
fig, ax=plt.subplots(nrows=1, ncols=2, figsize=(20,5), constrained_layout=True)
sns.kdeplot(x = 'purpose_by_income_type_SWITCHLOAN', data = not_applied, hue = 'label', shade = True, palette=['#BB3A32', '#8D6AB7'] , ax = ax[0])
sns.kdeplot(x = 'purpose_by_income_type_SWITCHLOAN', data = applied, hue = 'label', shade = True, palette=['#E58537', '#4A76AF', '#5A9D40'], ax = ax[1])
plt.rc('font', size=40)
#plt.rc('xtick', labelsize=30)  # x축 눈금 폰트 크기 
#plt.rc('ytick', labelsize=30)  # y축 눈금 폰트 크기
plt.show()

## 2. log_result 데이터에 대한 분석

In [ ]:
# log_result에 대한 대푯값 뽑기
pd.options.display.float_format = '{:.5f}'.format
pd.set_option('display.max_columns', None)
grouped_user= df.groupby(['label'])[['timedelta_max',
       'timedelta_min', 'timedelta_mean', 'timedelta_std',
       'timedelta_quantile_first', 'timedelta_quantile_second',
       'timedelta_quantile_third', 'CompleteIDCertification_count',
       'GetCreditInfo_count', 'UseDSRCalc_count', 'UseLoanManage_count',
       'UsePrepayCalc_count']]
grouped_user.agg(['mean', 'median']).reset_index()

In [ ]:
# log_result에 대한 대푯값 뽑기
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.max_columns', None)
grouped_user= df.groupby('label')[['CompleteIDCertification_count',
       'GetCreditInfo_count', 'UseDSRCalc_count', 'UseLoanManage_count',
       'UsePrepayCalc_count']]
grouped_user.agg(['min', 'max', 'mean', 'median']).reset_index()

In [116]:
# 1행 n열로 kdeplot 그리는 함수
def plot_kde(df, feature_names, ncols, start_idx):
    fig,ax=plt.subplots(nrows=1, ncols=ncols, figsize=(20,5))
    col_names=df[feature_names].columns
    for i in range(ncols):
        sns.kdeplot(data=df, x=col_names[i+start_idx], hue='label', palette=['#4A76AF', '#E58537', '#8D6AB7'], shade=True, ax=ax[i])
    plt.plot()

def plot_kde2(df, feature_names, ncols, start_idx):
    fig,ax=plt.subplots(nrows=1, ncols=ncols, figsize=(20,5))
    col_names=df[feature_names].columns
    for i in range(ncols):
        sns.kdeplot(data=df, x=col_names[i+start_idx], hue='label', palette=['#5A9D40','#BB3A32'], shade=True, ax=ax[i])
    plt.plot()

In [ ]:
#첫번째 
df_first = df.groupby('label')['is_applied_Y_first'].sum()
df_first

In [ ]:
df_last = df.groupby('label')['is_applied_Y_last'].sum()
df_last

In [ ]:
df2_last = df.groupby('label')['is_applied_Y_last'].sum()
df2_last

In [ ]:
df.groupby(['is_applied_Y_first']).sum()

In [ ]:
df['is_applied_Y_first'].head(30)

In [ ]:
plot_kde(applied, log_result, ncols=4, start_idx=0)
plot_kde2(not_applied, log_result, ncols = 4, start_idx=0)

In [ ]:
# log_result에 대한 대푯값 뽑기
pd.options.display.float_format = '{:.5f}'.format
pd.set_option('display.max_columns', None)
grouped_user= df.groupby(['label'])[['UseLoanManage_first']]
grouped_user.agg(['min', 'max']).reset_index()

####이 있는 고객들은 주로 대출 신청 전 #############(특정서비스이용) 행동을 수행함

In [ ]:
plot_kde(applied, log_result, ncols=4, start_idx=4)
plot_kde2(not_applied, log_result, ncols = 4, start_idx=4)

In [ ]:
plot_kde(applied, log_result, ncols=4, start_idx=8)
plot_kde2(not_applied, log_result, ncols=4, start_idx=8)

In [ ]:
plot_kde(applied, log_result, ncols=4, start_idx=12)
plot_kde2(not_applied, log_result, ncols=4, start_idx=12)

In [ ]:
plot_kde(applied, log_result, ncols=4, start_idx=16)
plot_kde2(not_applied, log_result, ncols=4, start_idx=16)

In [ ]:
plot_kde(applied, log_result, ncols=2, start_idx=20)
plot_kde2(not_applied, log_result, ncols=2, start_idx=20)

### ##########에 대한 분석. 더 크게 plot그려보기

In [ ]:
sns.kdeplot(x = 'UseLoanManage_first', data = not_applied, hue = 'label', shade = True, palette=['#BB3A32', '#8D6AB7'])
plt.savefig('../output/5_6_action_noapply.png')

In [ ]:
sns.kdeplot(x = 'UseLoanManage_first', data = applied, hue = 'label', shade = True, palette=['#E58537', '#4A76AF', '#5A9D40'])
plt.savefig('../output/5_7_action_apply.png')

In [ ]:
sns.kdeplot(x = 'UseLoanManage_last', data = not_applied, hue = 'label', shade = True, palette=['#BB3A32', '#8D6AB7'])
plt.savefig('../output/5_8_action_last_noapply.png')

In [ ]:
sns.kdeplot(x = 'UseLoanManage_last', data = applied, hue = 'label', shade = True, palette=['#E58537', '#4A76AF', '#5A9D40'])
plt.savefig('../output/5_9_action_last_apply.png')